### 1) Label 할 영상을 취득
### 2) 영상의 모든 Frame을 Get - [2] Extract Video
### 3) 모든 Frame 중 첫 번째 File 만 LabelImg 를 이용하여 labeling
### 4) 다른 Image의 Label을 자동으로 일괄 생성 (임의의 위치임) - [3] Clone xml files for rest of images
### 5) Labelimg 에서 해당 Folder 를 Load 하여 일괄 labeling 함
### 6) Label 한 image / xml 들을 Training model input size 에 맞도록 일괄 수정 - [4] Resize jpg files and mkae xmls 
### 7) Train / Test Data 로 Split 함.
\
### 여기서 더 할 것은, 영상의 모든 Frame 이 유효한 Frame 인 경우 (Label 이 필요한 경우) 그냥 하면 되지만
### 불필요한 (Label 이 필요없는) 쓰레기 Frame일 수 있으므로, 그 부분에 대한 고려가 필요함.
### 이 부분은, 일단 Video 를 통해서 image를 취득한 뒤 (위 [2] Extract Video) 
### 98_inferencing_and_make_labels.ipynb (기존 모델을 이용해서 자동 label을 하는 부분) 을 이용하여 하도록 함.

##### Test data 를 산출하기 위한 inferencing test 에서는 다음과 같이 수행하였음
##### 1) image extraction from video
##### 2) original image 를 기준으로 98_inferencing_and_make_lable_from_it.ipynb 이용하여
#####   2-1) score > 0.5 이상인 image 들만 valid 한 사진으로 분리
#####   2-2) valid 한 사진들을 머신러닝 모델로 자동 labeling
#####   2-3) valid 한 사진들이 진짜 valid 한지 labelimg 로 전부 확인.
#####   2-4) invalid 한 사진들이 진짜 invalid 한지 확인하고, valid 가 포함되 있는 경우 labelimg 로 손라벨하여 valid 에 포함시킴.
##### 3) valid 한 사진 및 이미지들을 [4] 이용하여 일괄 Resize (image, xml)
##### 4) 9_Cartucho_making_pre_datasets.ipynb 이용하여 제작된 이미지 / xml 로 
#####  4-1) detection-results : 머신러닝 모델로 detect 했을 때 나오는 좌표 저장
#####  4-2) ground-truth : xml 에 저장되어 있는 ground-truth 저장
##### 5) Cartucho 로 MAP 산정.
##### 단, 이렇게 하는 경우 invalid 한 image 들이 진짜 invalid 한지는 눈으로 확인하고 따져보아야 함.

In [ ]:
#### import numpy as np
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm
import sys
import math
import random
import shutil

class Input_Preprocess():
    def __init__(self):
        self.file_name = '13_Qmemo_typing_and_drawing'
#         self.video_file = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/1_video/calculator_test.mp4'
        self.video_file = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/1_video/Rainbow_Test/' + self.file_name + '.mp4'
        self.img_dir = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/Rainbow_Test/' + self.file_name + '/'
        self.resize_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/' + self.file_name + '/'
#         self.img_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/original_images/'
#         self.resize_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/resized_images/'

    def print_menu(self):
        print('==========================================================================================')
        print('[Input Preprocess] Input preprocessing options are as below')
        print('[1] Input video, img, resize dir')
        print('[2] Extract video (from video dir path) frames into jpg files (saving in image dir path)')
        print('[3] Clone xml files for rest of image files (same as first frame)')
        print('[4] Resize jpg files and make xmls for appropriate resolution of it')
        print('[5] Split Test & Train Datasets into train / test folder (default 0.9 percentage)')
        print('[9] Exit Program')
        print('==========================================================================================')
        print('[video file path] : %s' %(self.video_file))
        print('[image dir path]  : %s' %(self.img_dir))
        print('[saving dir path] : %s' %(self.resize_dir))
        print('==========================================================================================')

    def save_dir_path(self):
        self.video_file = input('Input video_file path (1example :  c:/..../~~.mp4): ')
        self.img_dir = input('Input img_dir path (where to save frames of videos & clone labels(xmls) of frames) : ')
        self.resize_dir = input('Input resize path (where to save resized frames and xmls of videos) : ')
        if not os.path.isfile(self.video_file):
            print('please input correct path where video is located in')
            
#             self.save_dir_path()
        if not os.path.exists(self.img_dir):
            os.mkdir(self.img_dir)
        if not os.path.exists(self.resize_dir):
            os.mkdir(self.resize_dir)

        if not self.img_dir.endswith('/'):
            self.img_dir += '/'
        if not self.resize_dir.endswith('/'):
            self.resize_dir += '/'
    
    def check_dir_validation(self):
        if not os.path.exists(self.img_dir):
            os.mkdir(self.img_dir)
        if not os.path.exists(self.resize_dir):
            os.mkdir(self.resize_dir)
            
    def extract_video(self):
        self.check_dir_validation()
        print(sys._getframe(0).f_code.co_name)
        video_handle = cv2.VideoCapture(self.video_file)
        i = 0
        while video_handle.isOpened():
            ret, frame = video_handle.read()
            if ret:
                if i == 0:
                    print(frame.shape)
#                 cv2.imshow('image', frame)
                cv2.imwrite(self.img_dir + 'images{0:04d}.jpg'.format(i+13000), frame)
#                 print('saving images %d .jpg' % i)
                i += 1
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break

        video_handle.release()
        cv2.destroyAllWindows()

    def clone_xml(self):
        self.check_dir_validation()
        print(sys._getframe(0).f_code.co_name)
        _dir = os.listdir(self.img_dir)
        file_list_jpg = []
        file_list_xml = []
        for file in _dir:
            file_splited = file.split('.')

            if file_splited[1].endswith('jpg'):
                file_list_jpg.append(file_splited[0])
            elif file_splited[1].endswith('xml'):
                file_list_xml.append(file_splited[0])
            else:
                print('weird!!! %s' % file)
        print('<< JPG Files : %d >>' % len(file_list_jpg))
        print('<< xml Files : %d >>' % len(file_list_xml))

        print('\n2. get base xml file and copy to all others')
        for i, file in enumerate(tqdm(file_list_jpg)):
            if i == 0:
                base_xml_file = self.img_dir + file + '.xml'
                if os.path.isfile(base_xml_file):
                    basexml = open(base_xml_file, 'rt', encoding='UTF8')
                    tree = ET.parse(basexml)
                    root = tree.getroot()

                    filename_tag = root.find('filename')
                    print('base_xml_file : ', filename_tag.text)
            else:
                edit_xml_file = self.img_dir + file + '.xml'
                filename_tag.text = file + '.jpg'
                tree.write(edit_xml_file)

    def resize_jpg_and_xml(self):
        print(sys._getframe(0).f_code.co_name)

        all_files = os.listdir(self.img_dir + 'valid')
        img_files = []
        xml_files = []
        for file in all_files:
            if file.endswith('jpg'):
                img_files.append(file)
            elif file.endswith('xml'):
                xml_files.append(file)

        # 1) Resize image files into new folder
        for i, file_name in enumerate(tqdm(img_files)):
            file = self.img_dir + 'valid/' + file_name
            if os.path.isfile(file):
                # 1) Image resize and write
                img = cv2.imread(file, cv2.IMREAD_COLOR)
                
                # print('file_name : %s, resolution (%d, %d, %d)' % (file_name, img.shape[0], img.shape[1], img.shape[2]))
                new_img = cv2.resize(img, dsize=(640, 640), interpolation=cv2.INTER_AREA)
#                 new_img = cv2.resize(img, dsize=(720, 1600), interpolation=cv2.INTER_AREA)

                new_file = self.resize_dir + file_name.split('.')[0] + '_resized_linear_0414.jpg'
#                 new_file = self.resize_dir + file_name.split('.')[0] + '_resized_linear_1600.jpg'

                cv2.imwrite(new_file, new_img)

        # 2) change xmls for new folder
        for i, file_name in enumerate(tqdm(xml_files)):
            file = self.img_dir + 'valid/' + file_name
            if os.path.isfile(file):

                # xml 수정하기
                targetXML = open(file, 'rt', encoding='UTF8')
                tree = ET.parse(targetXML)
                root = tree.getroot()

                # 이미지 이름 수정
                filename_tag = root.find("filename")
                filename_tag.text = str(file_name.split('.')[0] + '_resized_linear_0414.jpg')
#                 filename_tag.text = str(file_name.split('.')[0] + '_resized_linear_1600.jpg')

                # 이미지 사이즈 수정
                # origin width, target_width 를 가지고 좌표를 수정함.
                width_tag = root.find("size").find("width")
                origin_width = int(width_tag.text)
                target_width = 640
                width_tag.text = str(target_width)

                height_tag = root.find("size").find("height")
                origin_height = int(height_tag.text)
                target_height = 640
                height_tag.text = str(target_height)

                # 좌표수정
                for object in root.iter("object"):
                    xmin_tag = object.find("bndbox").find("xmin")
                    ymin_tag = object.find("bndbox").find("ymin")
                    xmax_tag = object.find("bndbox").find("xmax")
                    ymax_tag = object.find("bndbox").find("ymax")

                    org_xmin = int(xmin_tag.text)
                    org_ymin = int(ymin_tag.text)
                    org_xmax = int(xmax_tag.text)
                    org_ymax = int(ymax_tag.text)

                    xmin_tag.text = str(int(math.ceil(org_xmin * target_width / origin_width)))
                    ymin_tag.text = str(int(math.ceil(org_ymin * target_height / origin_height)))
                    xmax_tag.text = str(int(math.ceil(org_xmax * target_width / origin_width)))
                    ymax_tag.text = str(int(math.ceil(org_ymax * target_height / origin_height)))

                # print(file + "[success]")
                # xml 저장
                edit_xml_file = self.resize_dir + file_name.split('.')[0] + '_resized_linear_0414.xml'
                tree.write(edit_xml_file)

    # 90 percent : Training / 10 Percent : Test data
    def shuffle_and_save(self, split_rate=0.904):
        _dir = os.listdir(self.resize_dir)
        # print('rate = %f' %split_rate)
        print(_dir[:5], len(_dir))

        jpg_file = []
        for file in _dir:
            if file.endswith('jpg'):
                jpg_file.append(file)
        # print(jpg_file[:5], len(jpg_file))
        random.shuffle(jpg_file)
        print(jpg_file[:5], len(jpg_file))

        if not os.path.exists(self.resize_dir + 'test/'):
            os.mkdir(self.resize_dir + 'test/')
        if not os.path.exists(self.resize_dir + 'train/'):
            os.mkdir(self.resize_dir + 'train/')

        for i, file in enumerate(tqdm(jpg_file)):
            file_name = file.split('.')[0]
            if i < len(jpg_file)*split_rate:
                shutil.copy(self.resize_dir + file_name + '.jpg', self.resize_dir + 'train/')
                shutil.copy(self.resize_dir + file_name + '.xml', self.resize_dir + 'train/')
            else:
                shutil.copy(self.resize_dir + file_name + '.jpg', self.resize_dir + 'test/')
                shutil.copy(self.resize_dir + file_name + '.xml', self.resize_dir + 'test/')

        print('File Copy Done!')

    def activate(self, menu_num:int):
        print('received : ', menu_num)
        test = {1 : self.save_dir_path, 2 : self.extract_video, 3 : self.clone_xml, 4 : self.resize_jpg_and_xml, 5 : self.shuffle_and_save, 9 : sys.exit}
        return test[menu_num]()

def main():
    preprocess = Input_Preprocess()

    input_val = 0
    while input_val != 9:
        preprocess.print_menu()
        input_val = int(input('[Input Preprocess] Input menu number to activate preprocess : '))
        preprocess.activate(input_val)

if __name__ == '__main__':
    main()

[Input Preprocess] Input preprocessing options are as below
[1] Input video, img, resize dir
[2] Extract video (from video dir path) frames into jpg files (saving in image dir path)
[3] Clone xml files for rest of image files (same as first frame)
[4] Resize jpg files and make xmls for appropriate resolution of it
[5] Split Test & Train Datasets into train / test folder (default 0.9 percentage)
[9] Exit Program
[video file path] : /home001/yousung.choi/projects/project_coordination/tools/1_datasets/1_video/Rainbow_Test/13_Qmemo_typing_and_drawing.mp4
[image dir path]  : /home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/Rainbow_Test/13_Qmemo_typing_and_drawing/
[saving dir path] : /home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/13_Qmemo_typing_and_drawing/
[Input Preprocess] Input menu number to activate preprocess : 2
received :  2
extract_video
(1600, 720, 3)
[Input Preprocess] Input preprocessing options a

  2%|▏         | 5/270 [00:00<00:06, 43.79it/s]

received :  4
resize_jpg_and_xml


100%|██████████| 270/270 [00:00<00:00, 3522.77it/s]


[Input Preprocess] Input preprocessing options are as below
[1] Input video, img, resize dir
[2] Extract video (from video dir path) frames into jpg files (saving in image dir path)
[3] Clone xml files for rest of image files (same as first frame)
[4] Resize jpg files and make xmls for appropriate resolution of it
[5] Split Test & Train Datasets into train / test folder (default 0.9 percentage)
[9] Exit Program
[video file path] : /home001/yousung.choi/projects/project_coordination/tools/1_datasets/1_video/Rainbow_Test/13_Qmemo_typing_and_drawing.mp4
[image dir path]  : /home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/Rainbow_Test/13_Qmemo_typing_and_drawing/
[saving dir path] : /home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/13_Qmemo_typing_and_drawing/


In [13]:
video_dir = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/1_video/Rainbow_Test/'
img_dir = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/'
resize_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/'
# img_dir = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/Rainbow_Test/1_calculator/'
# resize_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/1_calculator/'

files = os.listdir(video_dir)
# print(files)

dir_needs = []
for file in files:
    new_dir = 'Rainbow_Test/' + file.split('.')[0]
#     print(new_dir)
    if not os.path.exists(img_dir + new_dir):
        os.mkdir(img_dir + new_dir)
    if not os.path.exists(resize_dir + new_dir):
        os.mkdir(resize_dir + new_dir)
    

# This is Test Code for preprocessing V30 images resolution like Rainbow Projects (720 x 1440 => 720 x 1600)

In [26]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm
import sys
import math
import random
import shutil

img_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/test_images/v30_original_images/'
save_dir = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/test_images/v30_resized_images/'
all_files = os.listdir(img_dir)

jpg_files = []
xml_files = []
for file in tqdm(all_files):
    if file.endswith('.jpg'):
        jpg_files.append(file)
    elif file.endswith('.xml'):
        xml_files.append(file)
        
# print(jpg_files[0])
# print(len(jpg_files))

jpg_files.sort()
xml_files.sort()

for file in tqdm(jpg_files):
    # 1) image resize and save
    img = cv2.imread(img_dir + file, cv2.IMREAD_COLOR)
    img = cv2.resize(img, dsize=(0, 0), fx=0.9, fy=0.9, interpolation=cv2.INTER_AREA)

    # # 이미지 범위 지정
    y, x, h, w = (0, 0, img.shape[0], img.shape[1])

    # # 그림 주변에 검은색으로 칠하기
    w_x = (720 - (w - x)) / 2  # w_x = (300 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
    h_y = (1600 - (h - y)) / 2

    if (w_x < 0):  # 크기가 -면 0으로 지정.
        w_x = 0
    elif (h_y < 0):
        h_y = 0

    M = np.float32([[1, 0, w_x], [0, 1, h_y]])  # (2*3 이차원 행렬)
    img_re = cv2.warpAffine(img, M, (720, 1600))

    img_re_h = img_re.shape[0]
    img_re_w = img_re.shape[1]
#     print("h:", img_re_h, " w:", img_re_w)

    # 이미지 저장하기
    edit_jpg_file = save_dir + file.split('.')[0] + '_edited.jpg'
    cv2.imwrite(edit_jpg_file, img_re)
    
# 2) xml info rewrite
for i, file_name in enumerate(tqdm(xml_files)):
        file = img_dir + file_name
        if os.path.isfile(file):

            # xml 수정하기
            targetXML = open(file, 'rt', encoding='UTF8')
            tree = ET.parse(targetXML)
            root = tree.getroot()

            # 이미지 이름 수정
            filename_tag = root.find("filename")
#                 filename_tag.text = str(file_name.split('.')[0] + '_resized_linear.jpg')
            filename_tag.text = str(file_name.split('.')[0] + '_edited.jpg')

            # 이미지 사이즈 수정
            # origin width, target_width 를 가지고 좌표를 수정함.
            width_tag = root.find("size").find("width")
            origin_width = int(width_tag.text)
            target_width = 720
            width_tag.text = str(target_width)

            height_tag = root.find("size").find("height")
            origin_height = int(height_tag.text)
            target_height = 1600
            height_tag.text = str(target_height)

            # 좌표수정
            for object in root.iter("object"):
                xmin_tag = object.find("bndbox").find("xmin")
                ymin_tag = object.find("bndbox").find("ymin")
                xmax_tag = object.find("bndbox").find("xmax")
                ymax_tag = object.find("bndbox").find("ymax")

                org_xmin = int(xmin_tag.text)
                org_ymin = int(ymin_tag.text)
                org_xmax = int(xmax_tag.text)
                org_ymax = int(ymax_tag.text)
                
                xmin_tag.text = str(int(math.ceil(org_xmin * 0.9 + (target_width * 0.05))))
                ymin_tag.text = str(int(math.ceil(org_ymin * 0.9 + (target_height - 1440 * 0.9)/2)))
                xmax_tag.text = str(int(math.ceil(org_xmax * 0.9 + (target_width * 0.05))))
                ymax_tag.text = str(int(math.ceil(org_ymax * 0.9 + (target_height - 1440 * 0.9)/2)))

#                 xmin_tag.text = str(int(math.ceil(org_xmin * target_width / origin_width)))
#                 ymin_tag.text = str(int(math.ceil(org_ymin * target_height / origin_height)))
#                 xmax_tag.text = str(int(math.ceil(org_xmax * target_width / origin_width)))
#                 ymax_tag.text = str(int(math.ceil(org_ymax * target_height / origin_height)))

            # print(file + "[success]")
            # xml 저장
            edit_xml_file = save_dir + file_name.split('.')[0] + '_edited.xml'
            tree.write(edit_xml_file)

100%|██████████| 1048/1048 [00:00<00:00, 4086.59it/s]


In [28]:
split_rate = 0.904
_dir = os.listdir(save_dir)
# print('rate = %f' %split_rate)
print(_dir[:5], len(_dir))

jpg_file = []
for file in _dir:
    if file.endswith('jpg'):
        jpg_file.append(file)
# print(jpg_file[:5], len(jpg_file))
random.shuffle(jpg_file)
print(jpg_file[:5], len(jpg_file))

if not os.path.exists(save_dir + 'test/'):
    os.mkdir(save_dir + 'test/')
if not os.path.exists(save_dir + 'train/'):
    os.mkdir(save_dir + 'train/')

for i, file in enumerate(tqdm(jpg_file)):
    file_name = file.split('.')[0]
    if i < len(jpg_file) * split_rate:
        shutil.copy(save_dir + file_name + '.jpg', save_dir + 'train/')
        shutil.copy(save_dir + file_name + '.xml', save_dir + 'train/')
    else:
        shutil.copy(save_dir + file_name + '.jpg', save_dir + 'test/')
        shutil.copy(save_dir + file_name + '.xml', save_dir + 'test/')

print('File Copy Done!')

 47%|████▋     | 492/1048 [00:00<00:00, 4913.96it/s]

['images0782_edited.xml', 'images0124_edited.jpg', 'images1428_edited.xml', 'images0445_edited.jpg', 'images0537_edited.jpg'] 2096
['images0154_edited.jpg', 'images0824_edited.jpg', 'images0252_edited.jpg', 'images1195_edited.jpg', 'images1971_edited.jpg'] 1048


100%|██████████| 1048/1048 [00:00<00:00, 4914.15it/s]

File Copy Done!
